# This is the code for empathy task in SIOP ML competition 2024
Author: Zihao Jia zjia2@gmu.edu


***Note: all the existing outputs in the current file were original outputs we used for submission.***

## Analysis strategy
###Step 1: Using GPT-4 to create a explanation why the original email demonstrated empathy. This is designed to provide a thought process for the GPT model for later classifications.
###Step 2: Using a few-shot learning GPT-4 model to classify empathy. During few shot learning, instead of only providing the original emails, we are also producing the reasoning process generated from step 1 to help model create a thinking process.

In [ ]:
# mount to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#install required pacakges
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.9/266.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.1 MB/s eta 0:00:00


In [ ]:
# set up API first
from openai import OpenAI
import openai
import os

client = OpenAI(api_key= 'your api')

In [ ]:
# load original train, eval, test sets
input_file_path = '/content/drive/MyDrive/2024 SIOP competition/Reproduce/data/empathy_train.csv'
eval_file_path = '/content/drive/MyDrive/2024 SIOP competition/Reproduce/data/empathy_val_public.csv'
test_file_path = '/content/drive/MyDrive/2024 SIOP competition/Reproduce/data/Copy of empathy_test_public.csv'

In [ ]:
# read the training file
import pandas as pd

# Replace 'your_file.csv' with the name of your CSV file
ds = pd.read_csv(input_file_path)

In [ ]:
# create two new columns in the training data to help generate the prompt used for reasoning later.
ds['label'] = ds['empathy'].apply(lambda x: 'yes' if x == 1 else 'no')
ds['did'] = ds['empathy'].apply(lambda x: 'did' if x == 1 else 'did not')

### Step 1: Generate explanations for training set data.
We have already saved and uploaded the new file with generated explanations. You are NOT required to run these codes if your purpose is to reproduce the results for competition only.

In [ ]:
# get the explanation for existing labels in training set.
# Specifically, I asked why the email demostrated empathy in the training set and save that in a new column named "reason"
ds['reason'] = None
for i in range(len(ds)):
  completion = client.chat.completions.create(
  model="gpt-4-0125-preview",
  temperature=1,
  seed = 1234,
  messages=[{"role": "user", "content": f"Here is an email written to Jonathan to provide some feedback to his work on the beta project ###Email starts###{ds.text[i]}###Email ends###. Though Jonathan did not do very well on the work, the writer still want to convey some empathy and understanding in this email to help Jonathan get improved. Now you think this email {ds.did[i]} demostrate empathy, why? Provide a short answer."}]
  )
  ds['reason'][i] = completion.choices[0].message.content

<ipython-input-15-bcccd9cc9866>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds['reason'][i] = completion.choices[0].message.content
<ipython-input-15-bcccd9cc9866>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds['reason'][i] = completion.choices[0].message.content
<ipython-input-15-bcccd9cc9866>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds['reason'][i] = completion.choices[0].message.content
<ipython-input-15-bcccd9c

In [ ]:
# an example of generated reasons
ds.reason[9]

"Yes, the email demonstrates empathy and understanding while providing feedback to Jonathan. The writer acknowledges Jonathan's excitement and willingness to develop into a team leader role, indicating a positive attitude towards his career aspirations. The feedback on Jonathan's need to improve business writing is delivered constructively, focusing on development areas without dismissing his efforts. Suggestions for improvement, such as focusing on sharp, concise reporting and getting inspired by internal reports, are provided to guide Jonathan. The writer also expresses a desire to discuss Jonathan's reports and development plan further, showing a genuine interest in his progress. The closing encouragement underscores the belief in Jonathan's potential to make a difference, reinforcing the message of support and confidence in his abilities."

In [ ]:
# we saved this file. So you can read it directly using the code in this block
#ds.to_csv('/content/drive/MyDrive/2024 SIOP competition/data/empathy_gpt4_reason.csv', index=False)
ds = pd.read_csv('/content/drive/MyDrive/2024 SIOP competition/Reproduce/data/empathy_gpt4_reason.csv')

### Step 2: few-shot learning with generated explanations.
make sure you have already update the 'ds' object with the codes in last block. Your 'ds' should contain a new column named 'reason' now.

In [ ]:
# we now read the test set.
ds_eval = pd.read_csv(test_file_path)

In [ ]:
# Because we want to inlucde all examples from training set (with explanations). We are aggregating them all together to serve as the few-shot learning examples later.
all_prompt_empa = []
for i in range(len(ds)):
  conversation = {"role": "user", "content": f"Here is an email written to Jonathan to provide some feedback to his work on the beta project.###Email starts###{ds.text[i]}###Email ends###. Though Jonathan did not do very well on the work, the writer still want to convey some empathy and understanding in this email to help Jonathan get improved. Do you think this email demostrate empathy? Please answer yes or no first, then provide a short reason."},{"role": "assistant", "content": f"{ds.label[i]}.{ds.reason[i]}"}
  all_prompt_empa.append(conversation)

In [ ]:
# Now we loop within the testing set to get the results
from itertools import chain
# create an empty predict object
predict = []
for i in range(len(ds_eval)):
  question = {"role": "user", "content": f"Here is an email written to Jonathan to provide some feedback to his work on the beta project.###Email starts###{ds_eval.text[i]}###Email ends###. Though Jonathan did not do very well on the work, the writer still want to convey some empathy and understanding in this email to help Jonathan get improved. Do you think this email demostrate empathy? Please answer yes or no first, you don't have to provide reason in this response."}
  all_prompt_empa_flat = list(chain.from_iterable(all_prompt_empa))   # these are all the few-shot learning examples from training set.
  all_prompt_empa_flat.append(question)                              # we are adding one more 'question' at the end of the few shot learning examples. This question is the prompt to get predicted results in test set.
  completion = client.chat.completions.create(
    model="gpt-4-0125-preview",
    temperature=0.8,
    messages=all_prompt_empa_flat,
    seed = 1234
  )
  predict.append(completion.choices[0].message.content)

In [ ]:
# here are our predicted results. (same as the results in final submissions)
predict

['yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'no',
 'yes',
 'no',
 'yes',
 'yes',
 'yes',
 'no',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'no',
 'yes',
 'yes',
 'yes',
 'yes',
 'no',
 'yes',
 'yes',
 'yes',
 'yes',
 'Yes',
 'yes',
 'yes',
 'yes',
 'no',
 'yes',
 'yes',
 'no',
 'yes',
 'yes',
 'no',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'no',
 'yes',
 'no',
 'yes',
 'yes',
 'Yes',
 'yes',
 'yes',
 'no',
 'yes',
 'yes',
 'no',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'no',
 'yes',
 'yes',
 'no',
 'no',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'no',
 'yes',
 'yes',
 'yes',
 'no',
 'yes',
 'no',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'no',
 'yes',
 'yes']